## $\color{#dda}{\text{1: Libs y fetch (pmdarima)}}$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import os
import pandas as pd
import pmdarima as pm
from pmdarima.utils import pacf, acf
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

core           INFO 	Loading data for Las Vegas Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 1 completed the race distance 00:00.001000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '11', '31', '18', '55', '44', '63', '14', '81', '10', 

## $\color{#dda}{\text{2: Creación de la serie de tiempo}}$

Acá tomamos solo a Checo y a Verstappen y convertimos sus sessiontime a segundos totales

## $\color{#dda}{\text{3: Visualizamos a ver qué hay}}$

## $\color{#dda}{\text{4: Bins}}$

## $\color{#dda}{\text{5: SARIMA}}$

## $\color{#dda}{\text{6: Error}}$

## $\color{#dda}{\text{7: Predicción últimas 5 vueltas completas}}$